In [5]:
import json
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

/Users/mattia/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# avvio un'istanza al sito

In [6]:
# Inizializza il servizio di ChromeDriver
service = Service(ChromeDriverManager().install())

# Inizializza il browser con il servizio configurato
session = webdriver.Chrome(service=service)

# Apri la pagina di login
login_url = 'https://learn.profession.ai/?msg=not-logged-in'
session.get(login_url)

"" eseguire ora il login manualmente tramite interfaccia ""

In [7]:
#Mi muovo nella pagina con i corsi
session.get("https://learn.profession.ai/profile")

# corsi mostrati nella prima pagina

### Ottengo i link dei corsi mostrati in pagina

In [8]:
# Trova il wrapper dei corsi
courses_wrapper = session.find_element(By.ID, 'coursesWrapper')

# Trova tutti i tag <a> all'interno del wrapper
course_links = courses_wrapper.find_elements(By.TAG_NAME, 'a')
active_link = []

# Estrai e stampa gli href per ogni link
for link in course_links:
    href_value = link.get_attribute('href')
    course_id = href_value.split('=')[-1]
    active_link.append(course_id)
    print(course_id)

httpslearnprofessionaicourseai-week-1
big-data
business-intelligence-con-powerbi
deep-learning-e-reti-neurali-artificiali
fondamenti-di-analisi-dati-con-excel
fondamenti-di-machine-learning
guida-completa-a-google-colab
analisi-dati-avanzata-con-excel
introduzione-master-data-analytics
introduzione-master-data-science
live
machine-learning-oltre-l-algoritmo
machine-learning-tecniche-avanzate
natural-language-processing
progetto-machine-learning-oltre-l-algoritmo
progetto-statistica-descrittiva
progetto-analisi-piloti-mondiale-f1
progetto-sql-per-la-data-science
progetto-fondamenti-di-analisi-dati-con-excel
analisi-dei-disastri-aerei-dal-1919-al-2023
progetto-analisi-di-big-data
progetto-tecniche-di-data-visualization
progetto-big-data
progetto-analisi-dati-con-excel
progetto-fondamenti-di-machine-learning
progetto-analisi-dati-avanzata-con-excel
progetto-machine-learning-tecniche-avanzate
progetto-deep-learning-e-reti-neurali
progetto-filtro-per-fake-news
progetto-business-intelligence

li pulisco e ne elimino alcuni

In [9]:
# Rimuovi la prima voce
active_link.pop(0)

# Rimuovi tutte le voci che contengono "progetto" o "live"
active_link = [link for link in active_link if "progetto" not in link and link != "live"]

# Stampa la lista filtrata
print(active_link)
len(active_link)

['big-data', 'business-intelligence-con-powerbi', 'deep-learning-e-reti-neurali-artificiali', 'fondamenti-di-analisi-dati-con-excel', 'fondamenti-di-machine-learning', 'guida-completa-a-google-colab', 'analisi-dati-avanzata-con-excel', 'introduzione-master-data-analytics', 'introduzione-master-data-science', 'machine-learning-oltre-l-algoritmo', 'machine-learning-tecniche-avanzate', 'natural-language-processing', 'analisi-dei-disastri-aerei-dal-1919-al-2023', 'programmazione-con-python', 'python-data-toolkit-numpy-pandas-pyplot', 'sql-per-la-data-science', 'statistica-descrittiva', 'statisitca-inferenziale', 'analisi-dati-con-excel', 'tecniche-di-data-visualization']


20

### qui prendo tutte le voci dei subchapter (deprecato)

In [ ]:
# Funzione per ottenere i link dei corsi
link = active_link

# Crea una lista per memorizzare tutti i corsi
all_courses_data = []

# Richiama session.get per ogni link
for link_to in link:
    url = f'https://learn.profession.ai/course/{link_to}'
    session.get(url)
    session.implicitly_wait(5)
    
    # Espandi tutte le sezioni cliccando i bottoni
    chapter_toggle_buttons = session.find_elements(By.CLASS_NAME, 'fas.fa-angle-down')
    for button in chapter_toggle_buttons:
        session.execute_script("arguments[0].scrollIntoView(true);", button)
        button.click()
        session.implicitly_wait(2)

    # Crea il JSON per ogni corso
    chapter_elements = session.find_elements(By.CLASS_NAME, 'lw-course-contents-chapters')
    course_data = []
    current_chapter = None

    # Regex per riconoscere il numero del capitolo
    chapter_pattern = re.compile(r'^\d{2}$')

    # Capitoli e sottocapitoli
    for chapter in chapter_elements:
        lines = chapter.text.strip().split('\n')

        for line in lines:
            line = line.strip()

            if chapter_pattern.match(line): 
                if current_chapter:
                    course_data.append(current_chapter)
                
                current_chapter = {
                    "title": "",
                    "sub_chapters": []
                }
            elif current_chapter and not current_chapter["title"]:
                current_chapter["title"] = line
            elif current_chapter:
                current_chapter["sub_chapters"].append(line)

    if current_chapter:
        course_data.append(current_chapter)

    # Struttura finale del corso
    course_json = {
        "course": link_to,
        "chapters": [
            {
                "title": chapter["title"],
                "sub_chapters": chapter["sub_chapters"]
            }
            for chapter in course_data
        ]
    }

    # Aggiungi il corso all'elenco generale dei corsi
    all_courses_data.append(course_json)

# Converte in JSON
formatted_json = json.dumps(all_courses_data, indent=4, ensure_ascii=False)
print(formatted_json)

with open('output_active_courses.json', 'w', encoding='utf-8') as f:
    json.dump(all_courses_data, f, ensure_ascii=False, indent=4)
    

### Qui prendo invece anche le icone

In [10]:
import re
import json

# Funzione per ottenere i link dei corsi
link = active_link

# Crea una lista per memorizzare tutti i corsi
all_courses_data = []

# Richiama session.get per ogni link
for link_to in link:
    url = f'https://learn.profession.ai/course/{link_to}'
    session.get(url)
    session.implicitly_wait(5)
    
    # Espandi tutte le sezioni cliccando i bottoni
    chapter_toggle_buttons = session.find_elements(By.CLASS_NAME, 'fas.fa-angle-down')
    for button in chapter_toggle_buttons:
        session.execute_script("arguments[0].scrollIntoView(true);", button)
        button.click() 
        session.implicitly_wait(2)

    # Trova i capitoli
    chapter_elements = session.find_elements(By.CLASS_NAME, 'lw-course-contents-chapter')
    course_data = []
    
    # Estrai i capitoli e sottocapitoli
    for chapter in chapter_elements:
        chapter_title = chapter.find_element(By.CLASS_NAME, 'learnworlds-heading3').text.strip()
        
        current_chapter = {
            "title": chapter_title,
            "sub_chapters": []
        }

        subchapter_elements = chapter.find_elements(By.CLASS_NAME, 'lw-course-contents-unit')

        for subchapter in subchapter_elements:
            # Prendi l'icona associata al sottocapitolo
            try:
                icon_element = subchapter.find_element(By.TAG_NAME, 'svg')
                icon_type = icon_element.get_attribute('data-icon')
            except:
                icon_type = 'other'

            # Prendi il testo del sottocapitolo
            try:
                subchapter_text = subchapter.find_element(By.CLASS_NAME, 'learnworlds-element').text.strip()
            except:
                subchapter_text = None

            # Aggiungi il sottocapitolo con l'icona
            if subchapter_text and icon_type:
                current_chapter["sub_chapters"].append({
                    "text": subchapter_text,
                    "icon": icon_type
                })

        course_data.append(current_chapter)

    # Struttura finale del corso
    course_json = {
        "course": link_to,
        "chapters": course_data
    }

    all_courses_data.append(course_json)

# Converte il risultato in JSON
formatted_json = json.dumps(all_courses_data, indent=4, ensure_ascii=False)
print(formatted_json)

# Salva
with open('output_active_courses_icon.json', 'w', encoding='utf-8') as f:
    json.dump(all_courses_data, f, ensure_ascii=False, indent=4)

[
    {
        "course": "big-data",
        "chapters": [
            {
                "title": "Introduzione ai Big Data",
                "sub_chapters": [
                    {
                        "text": "Introduzione e storia dei Big Data",
                        "icon": "video"
                    },
                    {
                        "text": "Materiale del corso",
                        "icon": "url"
                    },
                    {
                        "text": "Grandezze e dimensioni dei Big Data",
                        "icon": "video"
                    },
                    {
                        "text": "Sorgenti dei Big Data",
                        "icon": "video"
                    },
                    {
                        "text": "Le quattro V dei Big Data",
                        "icon": "video"
                    },
                    {
                        "text": "Analisi, casi d'uso e benefici",
              

#### modifichiamo il json per togliere tutto quello che non è video

In [11]:

json_string = formatted_json

# Funzione per filtrare il JSON prendo solamente i video e rimuovere "icon", mantenendo solo "text"
def filter_and_extract_text(json_data):
    for course in json_data:
        for chapter in course["chapters"]:
            chapter["sub_chapters"] = [sub_chapter["text"] for sub_chapter in chapter["sub_chapters"] if sub_chapter.get('icon') == 'video']
    return json_data



json_data = json.loads(json_string)

filtered_json_data = filter_and_extract_text(json_data)

# Output del JSON filtrato
filtered_json_string = json.dumps(filtered_json_data, indent=4)
print(filtered_json_string)

with open('output_active_courses_ONLY_video.json', 'w', encoding='utf-8') as f:
    json.dump(filtered_json_data, f, ensure_ascii=False, indent=4)

[
    {
        "course": "big-data",
        "chapters": [
            {
                "title": "Introduzione ai Big Data",
                "sub_chapters": [
                    "Introduzione e storia dei Big Data",
                    "Grandezze e dimensioni dei Big Data",
                    "Sorgenti dei Big Data",
                    "Le quattro V dei Big Data",
                    "Analisi, casi d'uso e benefici",
                    "Domande degli esperti di Big Data",
                    "Calcolo parallelo e distribuito"
                ]
            },
            {
                "title": "[Teoria] Le Tecnologie dei Big Data",
                "sub_chapters": [
                    "Le Tecnologie dei Big Data"
                ]
            },
            {
                "title": "[Teoria] Apache Spark",
                "sub_chapters": [
                    "Introduzione ad Apache Spark",
                    "Introduzione ad Apache Zeppelin"
                ]
            },

# ----------------------------------------- ----------------------------------------- -----------------------------------------

## VERSIONE ALTERNATIVA -> questa invece ottiene tutti i corsi in memoria del sito

In [ ]:
session.get("https://learn.profession.ai/profile")

In [ ]:
# Trova tutti gli elementi con la classe 'nav-course-title'
course_container= session.find_elements(By.ID, 'navCoursesContainer')

course_elements = session.find_elements(By.XPATH, '//*[@data-coursetitleid]')

course_titles = []

for element in course_elements:
    course_title = element.get_attribute('data-coursetitleid')
    course_titles.append(course_title)

for title in course_titles:
    print(f"{title}")

## VERSIONE ALTERNATIVA -> Questo invece ottiene tutti i corsi salvati in sessione come "course"

##### (non so, saranno diversi, ne mancano alcuni)

In [ ]:
session.get("https://learn.profession.ai/profile")

In [ ]:
# ottengo i corsi e i link
def get_course_link():
    link_to_values = []

    block_pattern = re.compile(r'"linkType":\s*"courseContentslink",.*?"linkTo":\s*"([^"]+)"', re.DOTALL)

    course_elements = session.find_elements(By.XPATH, '//script[contains(text(), "linkData")]')

    # Itera su ogni elemento per trovare 'linkTo' associati a 'courseContentslink'
    for element in course_elements:
        element_text = element.get_attribute('innerText')

        link_to_matches = block_pattern.findall(element_text)

        for match in link_to_matches:
            if match not in link_to_values:
                link_to_values.append(match)
            print(match)
    return link_to_values

In [ ]:
link = get_course_link()

all_courses_data = []

for link_to in link:
    url = f'https://learn.profession.ai/course/{link_to}'
    session.get(url)
    
    # Espandi tutte le sezioni cliccando i bottoni
    chapter_toggle_buttons = session.find_elements(By.CLASS_NAME, 'fas.fa-angle-down')
    for button in chapter_toggle_buttons:
        session.execute_script("arguments[0].scrollIntoView(true);", button)
        button.click() 

        session.implicitly_wait(2)

    # Crea il JSON per ogni corso
    chapter_elements = session.find_elements(By.CLASS_NAME, 'lw-course-contents-chapters')
    course_data = []
    current_chapter = None

    # Regex per riconoscere il numero del capitolo
    chapter_pattern = re.compile(r'^\d{2}$')

    # Estrai i capitoli e sottocapitoli
    for chapter in chapter_elements:
        lines = chapter.text.strip().split('\n')

        for line in lines:
            line = line.strip()

            if chapter_pattern.match(line):
                if current_chapter:
                    course_data.append(current_chapter)
                
                current_chapter = {
                    "title": "",  
                    "sub_chapters": []  
                }
            elif current_chapter and not current_chapter["title"]:
                current_chapter["title"] = line
            elif current_chapter:
                current_chapter["sub_chapters"].append(line)

    # Aggiungi l'ultimo capitolo se esiste
    if current_chapter:
        course_data.append(current_chapter)

    # Struttura finale del corso
    course_json = {
        "course": link_to,
        "chapters": [
            {
                "title": chapter["title"], 
                "sub_chapters": chapter["sub_chapters"]
            }
            for chapter in course_data
        ]
    }

    all_courses_data.append(course_json)

# Converte il risultato in JSON e lo stampa
formatted_json = json.dumps(all_courses_data, indent=4, ensure_ascii=False)
print(formatted_json)

with open('only_course.json', 'w', encoding='utf-8') as f:
    json.dump(all_courses_data, f, ensure_ascii=False, indent=4)